In [1]:
# %reload_ext autoreload
# %matplotlib notebook
# %load_ext autoreload
# %autoreload 2
# %reload_ext autoreload

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import seaborn as sns


In [2]:
m1 = 1.2
m0 = 1
scale = 1

n = 100

Y = np.random.binomial(1, .5, n)

X = Y * np.random.normal(m1, scale, n) + (1 - Y) * np.random.normal(m0, scale, n)

In [3]:
sns.distplot(X)

# Classical Result

In [4]:
import statsmodels.api as sm

lr1_sm = sm.Logit(Y, sm.tools.add_constant(X, prepend=False))
lr1_sm_result = lr1_sm.fit()



Optimization terminated successfully.
         Current function value: 0.686247
         Iterations 4


In [5]:
print lr1_sm_result.summary()

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  100
Model:                          Logit   Df Residuals:                       98
Method:                           MLE   Df Model:                            1
Date:                Tue, 14 Feb 2017   Pseudo R-squ.:                0.009955
Time:                        17:29:41   Log-Likelihood:                -68.625
converged:                       True   LL-Null:                       -69.315
                                        LLR p-value:                    0.2401
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.2289      0.197      1.163      0.245      -0.157       0.614
const         -0.2318      0.283     -0.819      0.413      -0.787       0.323


In [6]:
print lr1_sm_result.cov_params()

[[ 0.03870759 -0.03915428]
 [-0.03915428  0.08016219]]


# Bootstrap

In [7]:
NBS = 3000

wbs_all = []

for _ in xrange(NBS):
    ind_bs = np.random.choice(range(n), size=n, replace=True)
    lr_bs_sm = sm.Logit(Y[ind_bs], sm.tools.add_constant(X[ind_bs], prepend=False))
    lr_bs_sm_result = lr_bs_sm.fit(disp=False)
    wbs_all.append(lr_bs_sm_result.params.tolist())



In [8]:
wbs_all = np.array(wbs_all)

print np.cov(wbs_all.T)

[[ 0.04728843 -0.04724364]
 [-0.04724364  0.09065815]]


In [9]:
sns.jointplot(wbs_all[:,0], wbs_all[:,1], kind='scatter')

In [10]:
plt.figure()
plt.plot(wbs_all[:,0], wbs_all[:,1], '.')
plt.axes().set_aspect('equal', 'datalim')

# Stochastic Inverse Hessian Gradient Descent

In [11]:
from __future__ import division

import math

def logistic_1d_hessian_sgd(X, Y, learn_rate, batch_size, 
                            burn_in, sample_intv, n_sample, 
                            hessian_batch, hessian_intv):
    """
    hessian_batch - number of samples used to compute hessain
    hessian_intv - number of steps waited till hessian is updated
    """
    
    w = np.zeros((2, 1))
    X = sm.tools.add_constant(X, prepend=False)
    
    n = len(Y)
    Y = 2 * Y - 1
    
    w_all = []
    
    for isgd in xrange(burn_in + sample_intv * n_sample + 4):
        if isgd % hessian_intv == 0:
            H = np.zeros((2, 2))
            indH = np.random.choice(range(n), size=hessian_batch, replace=False)
            for iH in indH:
                Xi = X[iH, :].reshape((-1, 1))
                tmp1_dp = float(w.T.dot(Xi)[0])
                tmp2 = np.exp(tmp1_dp - 2 * np.logaddexp(0, tmp1_dp))
                H = H + tmp2 * (Xi.dot(Xi.T))
            H = H / hessian_batch
            Hinv = np.linalg.inv(H)
        
        ind_batch = np.random.choice(range(n), size=batch_size, replace=True)
        
        Xb = X[ind_batch, :]
        Yb = Y[ind_batch]
        
        tmp1_dp = Xb.dot(w).ravel() * Yb
        
        tmp2 = np.exp(-np.logaddexp(np.zeros((batch_size,)), tmp1_dp))
        
        sgd = np.zeros((2, 1))
        for ib in xrange(batch_size):
            sgd = sgd - Xb[ib, :].reshape((-1, 1)) * tmp2[ib] * Yb[ib]
        sgd = sgd / batch_size
        
        w = w - learn_rate * Hinv.dot(sgd)
        
        if isgd > burn_in and (isgd - burn_in) % sample_intv == 1:
            w_all.append(w.ravel().tolist())
    
    return np.array(w_all)


In [12]:
# learn_rate = 0.01
# batch_size = 50
# burn_in = 1000
# sample_intv = 500
# n_sample = 100
# hessian_batch = 50
# hessian_intv = 20

# w_sgd = logistic_1d_hessian_sgd(X, Y, learn_rate, batch_size, 
#                             burn_in, sample_intv, n_sample, 
#                             hessian_batch, hessian_intv)


In [13]:
# print np.mean(w_sgd, axis=0)

In [14]:
# print np.cov(w_sgd.T) * (2 * batch_size / learn_rate / n)

In [15]:
# sns.jointplot(w_sgd[:,0], w_sgd[:,1], kind='scatter')

In [16]:
# plt.figure()
# plt.plot(w_sgd[:,0], w_sgd[:,1], '.')
# plt.axes().set_aspect('equal', 'datalim')

# averaged SGD

In [17]:
import tensorflow as tf

In [18]:
from __future__ import division

import tensorflow as tf

import random



def sgd_avg_infer(X_, Y_, lr, 
                  size_batch, 
                  burn_in, n_avg, n_skip, n_sample):
    lr = float(lr)
    
    n, p = X_.shape
    
    
    
    
    X = tf.placeholder(tf.float64, [None, 2])
    
    Y = tf.placeholder(tf.float64, [None, 1])
    
    w = tf.Variable(initial_value=tf.zeros([p, 1], dtype=tf.float64), dtype=tf.float64)

    log_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(tf.matmul(X, w), Y))
    
    opt = tf.train.GradientDescentOptimizer(lr).minimize(log_loss)
    
    init = tf.global_variables_initializer()
    
    
    
    n_iter = burn_in + (n_avg + n_skip) * n_sample + 3
    
    
    w_avg_samples = []
    
    w_all = []
    
    with tf.Session() as sess:
        sess.run(init)
        
        for i_ in xrange(n_iter):
            ind_batch = np.random.choice(range(n), size=size_batch, replace=True)
            
            Xb = X_[ind_batch, :].reshape((-1, p))
            Yb = Y_[ind_batch].reshape((-1, 1))
            
            sess.run([opt], feed_dict={X: Xb, Y: Yb})
            
            if i_ >= burn_in:
                if (i_ - burn_in) % (n_avg + n_skip) < n_avg:
                    wb = w.eval(session=sess).ravel().tolist()
                    w_avg_samples.append(wb)
                elif (i_ - burn_in) % (n_avg + n_skip) == n_avg:
                    w_avg_samples = np.array(w_avg_samples)
                    w_avg = np.mean(w_avg_samples, axis=0)
                    w_all.append(w_avg.ravel().tolist())
                    w_avg_samples = []
    
    return np.array(w_all)
    
    

In [19]:


# burn_in = 10000

# n_avg = 1000
# n_skip = 10
# n_sample = 1000

# lr = 0.1

# size_batch = 500


# w_sgd_avg = sgd_avg_infer(sm.tools.add_constant(X, prepend=False), Y, lr, size_batch, burn_in, n_avg, n_skip, n_sample)




In [20]:
# print np.mean(w_sgd_avg, axis=0)

# print np.cov(w_sgd_avg.T)

# print np.cov(w_sgd_avg.T) * size_batch / n * n_avg

In [21]:
# sns.jointplot(w_sgd_avg[:,0], w_sgd_avg[:,1], kind='scatter')

In [22]:
# plt.figure()
# plt.plot(w_sgd_avg[:,0], w_sgd_avg[:,1], '.')
# plt.axes().set_aspect('equal', 'datalim')

# logistic square

In [23]:
from __future__ import division

import tensorflow as tf

import random



def logistic_square(X_, Y_, lr, 
                    size_batch, 
                    burn_in, n_avg, n_skip, n_sample):
    lr = float(lr)
    
    n, p = X_.shape
    
    
    
    
    X1 = tf.placeholder(tf.float64, [None, 2])
    
    Y1 = tf.placeholder(tf.float64, [None, 1])
    
    w = tf.Variable(initial_value=tf.zeros([p, 1], dtype=tf.float64), dtype=tf.float64)

    log_loss1 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(tf.matmul(X1, w), Y1)) + 1
    
    X2 = tf.placeholder(tf.float64, [None, 2])
    
    Y2 = tf.placeholder(tf.float64, [None, 1])
    
    log_loss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(tf.matmul(X2, w), Y2)) + 1
    
    #opt = tf.train.GradientDescentOptimizer(lr).minimize(log_loss1 * log_loss2)
    
    
    opt = tf.train.GradientDescentOptimizer(learning_rate=-1.0)
    
    log_loss1_grads_and_vars = opt.compute_gradients(log_loss1, [w])
    
    grad_log_loss1 = log_loss1_grads_and_vars[0][0]
    
    
    w_next = tf.mul(tf.constant(-lr, dtype=tf.float64), grad_log_loss1) * log_loss2 + log_loss1_grads_and_vars[0][1]
    
    
    opt_op = opt.apply_gradients([(w_next - log_loss1_grads_and_vars[0][1], log_loss1_grads_and_vars[0][1])])
    
    
    init = tf.global_variables_initializer()
    
    
    
    n_iter = burn_in + (n_avg + n_skip) * n_sample + 3
    
    
    w_avg_samples = []
    
    w_all = []
    
    with tf.Session() as sess:
        sess.run(init)
        
        for i_ in xrange(n_iter):
            ind_batch1 = np.random.choice(n, size=size_batch, replace=True)
            
            Xb1 = X_[ind_batch1, :].reshape((-1, p))
            Yb1 = Y_[ind_batch1].reshape((-1, 1))
            
            ind_batch2 = np.random.choice(n, size=min(size_batch, n), replace=False)
            Xb2 = X_[ind_batch2, :].reshape((-1, p))
            Yb2 = Y_[ind_batch2].reshape((-1, 1))
            
            sess.run([opt_op], feed_dict={X1: Xb1, Y1: Yb1, X2: Xb2, Y2: Yb2})
            
            if i_ >= burn_in:
                if (i_ - burn_in) % (n_avg + n_skip) < n_avg:
                    wb = w.eval(session=sess).ravel().tolist()
                    w_avg_samples.append(wb)
                elif (i_ - burn_in) % (n_avg + n_skip) == n_avg:
                    w_avg_samples = np.array(w_avg_samples)
                    w_avg = np.mean(w_avg_samples, axis=0)
                    w_all.append(w_avg.ravel().tolist())
                    w_avg_samples = []
    
        w_all = np.array(w_all)
    
        w_opt = np.mean(w_all, axis=0)
        
        w.assign(w_opt.reshape((-1, 1)))
    
        f_opt = float(sess.run(log_loss1, feed_dict={X1: X_.reshape((-1, p)), Y1: Y_.reshape((-1, 1))}))
    
    return w_all, f_opt
    
    



In [24]:


burn_in = 1000

n_avg = 1000
n_skip = 10
n_sample = 1000

lr = 0.1

size_batch = 1000


w_logsq, f_opt = logistic_square(sm.tools.add_constant(X, prepend=False), Y, lr, size_batch, burn_in, n_avg, n_skip, n_sample)




In [25]:
print np.mean(w_logsq, axis=0)

print np.cov(w_logsq.T)

print np.cov(w_logsq.T) * size_batch / n * n_avg 






[ 0.22885192 -0.23179654]
[[  3.69358041e-06  -3.61768744e-06]
 [ -3.61768744e-06   7.31101835e-06]]
[[ 0.0369358  -0.03617687]
 [-0.03617687  0.07311018]]


In [26]:
print f_opt

1.68625403208


In [27]:
sns.jointplot(w_logsq[:,0], w_logsq[:,1], kind='scatter')

In [28]:
plt.figure()
plt.plot(w_logsq[:,0], w_logsq[:,1], '.')
plt.axes().set_aspect('equal', 'datalim')